In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/mbti_1.csv')
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
df_raw = data.copy()
df_raw.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
pip install texthero

In [ ]:
import texthero as hero
custom_pipeline = [hero.preprocessing.fillna,
                   hero.preprocessing.lowercase,
                   hero.preprocessing.remove_whitespace,
                   hero.preprocessing.remove_urls,
                   hero.preprocessing.remove_digits,
                   hero.preprocessing.remove_punctuation,
                   hero.preprocessing.remove_diacritics
                   ]

df_raw['clean_posts_nn'] = df_raw['posts'].pipe(hero.clean, custom_pipeline)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def clear_text(data):
    data_length=[]
    lemmatizer=WordNetLemmatizer()
    stemmer = PorterStemmer()
    cachedStopWords = stopwords.words("english")
    cleaned_text=[]
    for sentence in data.clean_posts_nn:
        sentence=sentence.lower()
        
# #         removing links from text data
#         sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    
# #         removing other symbols Keep only words
#         sentence=re.sub('[^0-9a-z]',' ',sentence)
# #          remove spaces > 1
#         sentence = re.sub(' +', ' ', sentence)
#          Remove stopwords and lematize
        sentence = " ".join([lemmatizer.lemmatize(w) for w in sentence.split(' ') if w not in cachedStopWords])
        
        data_length.append(len(sentence.split()))
        cleaned_text.append(sentence)
    return cleaned_text,data_length

In [ ]:
df_pro = df_raw.copy()

In [ ]:
df_pro = df_pro.drop(columns='posts')

In [ ]:
df_pro.clean_posts_nn,train_length=clear_text(df_pro)
df_pro.head()

,type,clean_posts_nn
0,INFJ,intj moment sportscenter top ten play p...
1,ENTP,finding lack post alarming sex boring positio...
2,INTP,good one course say know blessing curse...
3,INTJ,dear intp enjoyed conversation day esoteric...
4,ENTJ,fired another silly misconception approachin...


In [ ]:
df_pro['type'].value_counts()

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X = df_pro['clean_posts_nn'] # features
y = df_pro['type']  # labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.2)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer()
# X_train_tfidf = vectorizer.fit_transform(X_train)
    
# clf = LinearSVC()
# clf.fit(X_train_tfidf, y_train)
    
# Pipelining the vectorizer and the classifier
text_lr = Pipeline([('tfidf',TfidfVectorizer()),('lr',LogisticRegression())])
text_lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lr', LogisticRegression())])

In [ ]:
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import accuracy_score

In [ ]:
lr_predictions = text_lr.predict(X_test)
print(classification_report(y_test, lr_predictions))

              precision    recall  f1-score   support

        ENFJ       0.80      0.11      0.19        38
        ENFP       0.73      0.57      0.64       135
        ENTJ       0.91      0.22      0.35        46
        ENTP       0.74      0.58      0.65       137
        ESFJ       0.00      0.00      0.00         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.00      0.00      0.00        18
        INFJ       0.65      0.70      0.68       294
        INFP       0.60      0.89      0.72       366
        INTJ       0.61      0.64      0.62       218
        INTP       0.60      0.80      0.68       261
        ISFJ       0.83      0.15      0.26        33
        ISFP       0.81      0.24      0.37        54
        ISTJ       0.80      0.20      0.31        41
        ISTP       0.77      0.30      0.43        67

    accuracy                           0.63      1735
   macro avg       0.55   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('Testing accuracy %s' % accuracy_score(y_test, lr_predictions))
print('Testing F1 score: {}'.format(f1_score(y_test, lr_predictions, average='weighted')))

Testing accuracy 0.6340057636887608
Testing F1 score: 0.60196845938737


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
text_rf = Pipeline([('tfidf',TfidfVectorizer()),('rf',RandomForestClassifier())])
text_rf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('rf', RandomForestClassifier())])

In [ ]:
rf_predictions = text_rf.predict(X_test)

print(classification_report(y_test, rf_predictions))

              precision    recall  f1-score   support

        ENFJ       0.00      0.00      0.00        38
        ENFP       0.80      0.06      0.11       135
        ENTJ       0.00      0.00      0.00        46
        ENTP       0.71      0.09      0.16       137
        ESFJ       0.00      0.00      0.00         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.00      0.00      0.00        18
        INFJ       0.41      0.50      0.45       294
        INFP       0.33      0.88      0.48       366
        INTJ       0.49      0.25      0.33       218
        INTP       0.47      0.47      0.47       261
        ISFJ       0.00      0.00      0.00        33
        ISFP       0.00      0.00      0.00        54
        ISTJ       0.00      0.00      0.00        41
        ISTP       1.00      0.01      0.03        67

    accuracy                           0.38      1735
   macro avg       0.26   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('Testing accuracy %s' % accuracy_score(y_test, rf_predictions))
print('Testing F1 score: {}'.format(f1_score(y_test, rf_predictions, average='weighted')))

Testing accuracy 0.38443804034582135
Testing F1 score: 0.3115493657143083


## XGB



In [ ]:
from xgboost import XGBClassifier
text_xgb = Pipeline([('tfidf',TfidfVectorizer()),('xgb',XGBClassifier())])
text_xgb.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('xgb', XGBClassifier(objective='multi:softprob'))])

In [ ]:
xgb_predictions = text_xgb.predict(X_test)

print(classification_report(y_test, xgb_predictions))

              precision    recall  f1-score   support

        ENFJ       0.47      0.21      0.29        38
        ENFP       0.68      0.67      0.67       135
        ENTJ       0.70      0.46      0.55        46
        ENTP       0.65      0.67      0.66       137
        ESFJ       1.00      0.22      0.36         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.64      0.50      0.56        18
        INFJ       0.70      0.71      0.71       294
        INFP       0.68      0.83      0.75       366
        INTJ       0.67      0.65      0.66       218
        INTP       0.67      0.76      0.71       261
        ISFJ       0.67      0.55      0.60        33
        ISFP       0.62      0.39      0.48        54
        ISTJ       0.81      0.63      0.71        41
        ISTP       0.73      0.60      0.66        67

    accuracy                           0.68      1735
   macro avg       0.61   

In [ ]:
print('Testing accuracy %s' % accuracy_score(y_test, xgb_predictions))
print('Testing F1 score: {}'.format(f1_score(y_test, xgb_predictions, average='weighted')))

Testing accuracy 0.6801152737752162
Testing F1 score: 0.6700825975535711


### LightGBM

In [ ]:
import lightgbm as lgb
text_lg = Pipeline([('tfidf',TfidfVectorizer()),('lg',lgb.LGBMClassifier())])
text_lg.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lg', LGBMClassifier())])

In [ ]:
lg_predictions = text_lg.predict(X_test)

print(classification_report(y_test, lg_predictions))

              precision    recall  f1-score   support

        ENFJ       0.42      0.13      0.20        38
        ENFP       0.68      0.65      0.66       135
        ENTJ       0.76      0.35      0.48        46
        ENTP       0.66      0.69      0.68       137
        ESFJ       1.00      0.22      0.36         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       1.00      0.12      0.22         8
        ESTP       0.50      0.22      0.31        18
        INFJ       0.66      0.70      0.68       294
        INFP       0.67      0.83      0.74       366
        INTJ       0.64      0.65      0.65       218
        INTP       0.65      0.77      0.71       261
        ISFJ       0.64      0.42      0.51        33
        ISFP       0.64      0.33      0.44        54
        ISTJ       0.84      0.51      0.64        41
        ISTP       0.75      0.57      0.64        67

    accuracy                           0.67      1735
   macro avg       0.66   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('Testing accuracy %s' % accuracy_score(y_test, lg_predictions))
print('Testing F1 score: {}'.format(f1_score(y_test, lg_predictions, average='weighted')))

Testing accuracy 0.6651296829971182
Testing F1 score: 0.650925083130257


### SVC

In [ ]:

from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [ ]:
clf_predictions = text_clf.predict(X_test)

print(classification_report(y_test, clf_predictions))

              precision    recall  f1-score   support

        ENFJ       0.73      0.29      0.42        38
        ENFP       0.72      0.66      0.69       135
        ENTJ       0.80      0.43      0.56        46
        ENTP       0.68      0.72      0.70       137
        ESFJ       1.00      0.33      0.50         9
        ESFP       1.00      0.10      0.18        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.55      0.33      0.41        18
        INFJ       0.66      0.67      0.67       294
        INFP       0.68      0.84      0.75       366
        INTJ       0.61      0.61      0.61       218
        INTP       0.65      0.74      0.69       261
        ISFJ       0.73      0.58      0.64        33
        ISFP       0.72      0.48      0.58        54
        ISTJ       0.72      0.44      0.55        41
        ISTP       0.69      0.52      0.59        67

    accuracy                           0.67      1735
   macro avg       0.68   

In [ ]:
print('Testing accuracy %s' % accuracy_score(y_test, clf_predictions))
print('Testing F1 score: {}'.format(f1_score(y_test, clf_predictions, average='weighted')))

Testing accuracy 0.667435158501441
Testing F1 score: 0.6582035311247844


### Multinominal Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
text_nb = Pipeline([('tfidf',TfidfVectorizer()),('nb',MultinomialNB())])
text_nb.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('nb', MultinomialNB())])

In [ ]:
nb_predictions = text_nb.predict(X_test)

print(classification_report(y_test, nb_predictions))

              precision    recall  f1-score   support

        ENFJ       0.00      0.00      0.00        38
        ENFP       0.00      0.00      0.00       135
        ENTJ       0.00      0.00      0.00        46
        ENTP       0.00      0.00      0.00       137
        ESFJ       0.00      0.00      0.00         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.00      0.00      0.00         8
        ESTP       0.00      0.00      0.00        18
        INFJ       0.67      0.01      0.01       294
        INFP       0.21      1.00      0.35       366
        INTJ       0.00      0.00      0.00       218
        INTP       1.00      0.00      0.01       261
        ISFJ       0.00      0.00      0.00        33
        ISFP       0.00      0.00      0.00        54
        ISTJ       0.00      0.00      0.00        41
        ISTP       0.00      0.00      0.00        67

    accuracy                           0.21      1735
   macro avg       0.12   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('Testing accuracy %s' % accuracy_score(y_test, nb_predictions))
print('Testing F1 score: {}'.format(f1_score(y_test, nb_predictions, average='weighted')))

Testing accuracy 0.21268011527377523
Testing F1 score: 0.07706721557589061
